In [1]:
import pandas as pd
import requests
import time

In [1]:
# Riot Developer Portal에서 받은 API KEY입니다.
# 해당 값을 포함한 모든 종류의 KEY는 코드에 직접적으로 노출되지 않도록 하는 것이 좋습니다.
api_key = 'api key'

# 소환사명입니다.
summoner_name = 'summoner_name'

In [2]:
# SUMMONER-V4 API URL
requesturl = 'https://kr.api.riotgames.com/lol/summoner/v4/summoners/by-name/Ma3str0?api_key=RGAPI-25097981-756b-4175-8ded-3a178463a7c4'

In [3]:
# puuid입니다.
summoner_puuid = 'lV4UO9-b2g7Eq5SoIzSeghEyMAzIQInpz_ANvNbNsQ8bEgJu-oMQ2s896-6ccxRhlRoRZ4uad78CNQ'

In [4]:
games_list = []

In [5]:
# 솔랭 게임 목록 가져오기

games_list = []
index = 0
while True:
    requesturl = 'https://asia.api.riotgames.com/lol/match/v5/matches/by-puuid/lV4UO9-b2g7Eq5SoIzSeghEyMAzIQInpz_ANvNbNsQ8bEgJu-oMQ2s896-6ccxRhlRoRZ4uad78CNQ/ids?startTime=420&start=0&count=100&api_key=RGAPI-25097981-756b-4175-8ded-3a178463a7c4'
    
    while True:
        response = requests.get(requesturl)
        if response.ok:
            game_list_new = response.json()
            break
    
    games_list.extend(game_list_new)
    
    if len(game_list_new) != 100:
        break
        
    index += 100
    time.sleep(1)

NameError: name 'requests' is not defined

In [15]:
# 솔랭 게임 각각에 대해서 API 호출 및 row binding
games_df = pd.DataFrame()
for matchId in games_list:
    requesturl = 'https://asia.api.riotgames.com/lol/match/v5/matches/KR_5521870655?api_key=RGAPI-25097981-756b-4175-8ded-3a178463a7c4'
    
    
    while True:
        response = requests.get(requesturl)
        if response.ok:
            game_json = response.json()
            break
    game_df = pd.DataFrame(game_json['info']['participants'])[['puuid','championId','win','kills','deaths','assists']]
    game_df = game_df[game_df.puuid == summoner_puuid][['championId','win','kills','deaths','assists']]
        
    games_df = pd.concat([games_df,game_df])
    time.sleep(1)

KeyboardInterrupt: 

In [ ]:
pd.DataFrame(game_json['info']['participants'])[['puuid','championId','win','kills','deaths','assists']]

In [ ]:
games_df = games_df.groupby(['championId'],as_index=False).\
apply(lambda x: pd.Series({'win_count':sum(x.win),
                           'count':len(x),
                           'kills':sum(x.kills),
                           'deaths':sum(x.deaths),
                           'assists':sum(x.assists)
                          }))

In [ ]:
# 챔피언 Key값에 대한 Value 가져오기
last_version = requests.get("https://ddragon.leagueoflegends.com/api/versions.json").json()[0]
champion_constant = requests.get("http://ddragon.leagueoflegends.com/cdn/"+last_version+"/data/ko_KR/champion.json")

In [ ]:
# Json 파일을 DataFrame으로 변환
champion_df = pd.DataFrame(champion_constant.json()['data']).T[['key','name']]

# 변수형 문자 -> 숫자
champion_df['key'] = pd.to_numeric(champion_df['key'])

In [ ]:
games_df = games_df.merge(champion_df,
                          how = 'left',
                          left_on = 'championId',
                          right_on = 'key'
                         )[['name','count','win_count','kills','deaths','assists']]

In [ ]:
games_df['WR'] = games_df['win_count'] / games_df['count']
games_df['KDA'] = (games_df['kills']+games_df['assists']) / games_df['deaths']

In [ ]:
games_df.sort_values('count',ascending = False)[['name','WR','count','KDA']]